## Requirements

In [3]:
import datetime
import pandas as pd
import polars as pl

## Reading CSV data

The file `large_data_0001.csv` is not part of this repository due to its size.  It can be generated using the `create_csv_data.py` script.  However, note that this takes a couple of minutes for 2,500,000 rows and 100 columns.

In [4]:
file_name = 'large_data_0001.csv'

In [7]:
%time df_polars = pl.read_csv('large_data_0001.csv', try_parse_dates=True)

CPU times: user 3.48 s, sys: 1.75 s, total: 5.22 s
Wall time: 890 ms


In [8]:
%time df_pandas = pd.read_csv(file_name, parse_dates=['timestamp',])

CPU times: user 8.52 s, sys: 2.33 s, total: 10.9 s
Wall time: 10.8 s


Note that polars is clearly much faster and using multiple threads as the CPU time exceeds the walltime by a factor of 4.

In [10]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788323 entries, 0 to 788322
Columns: 101 entries, timestamp to C100
dtypes: datetime64[ns](1), float64(100)
memory usage: 607.5 MB


In [11]:
df_polars.estimated_size()

636964984

The size of the dataframes in memory is comparable.

The data consists of a single column with timestamps, and 100 columns of random double precision floating point values.

## Group by

To compare performance

In [24]:
days_pandas = df_pandas.groupby(df_pandas.timestamp.dt.day).mean().drop('timestamp', axis=1)

In [25]:
days_pandas

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
timestamp,,,,,,,,,,,,,,,,,,,,,
8,1.616301,1.300036,0.817093,-0.913630,0.535643,-0.429099,1.379245,1.250309,0.573959,1.237401,...,1.028077,0.495149,0.427012,1.460489,1.121102,0.547403,0.544154,0.869386,0.418865,0.095361
9,1.786842,-0.056498,1.201573,-3.228591,0.872878,-1.875540,0.358851,1.010238,0.715874,0.827519,...,-0.706015,-1.444457,2.049320,3.110629,0.452043,0.017991,1.616743,2.704365,1.580477,0.372258
10,1.166582,-1.765113,2.616493,-4.584708,0.898354,-1.781852,-1.061848,3.123990,-1.051368,0.092403,...,-0.666805,-0.931251,5.258575,4.489650,0.336976,-2.257478,2.266737,3.005411,0.266224,0.728347
11,0.684858,0.075567,3.814915,-5.174099,-1.534156,-0.894954,-2.950941,6.056719,-2.593477,0.549919,...,-0.150487,-0.922968,4.638803,6.565021,-2.114639,-2.469373,2.944349,1.892101,0.343170,-0.247440
12,1.235231,1.158072,3.888712,-3.935502,-1.398850,0.489980,-3.206239,6.448509,-3.039776,-2.741378,...,2.216628,-0.647440,3.961334,8.398948,-4.383207,-1.800691,1.611947,0.446160,1.459684,1.974048
13,2.143065,0.349599,2.831058,-4.061167,-0.497423,2.322625,-3.918994,5.501107,-4.203526,-4.138458,...,4.040739,0.269732,2.153507,8.603117,-3.973901,-2.322601,1.501349,-0.178385,1.480074,3.288867
14,3.689370,0.905805,1.035925,-2.450461,0.197875,3.995446,-4.549313,6.753407,-5.230918,-5.653404,...,3.530662,1.607023,-1.462408,6.047800,-6.236200,-4.593256,1.584021,-0.868583,3.156440,3.118407
15,5.932281,2.894121,-0.157428,0.432282,1.267441,3.859335,-6.094502,5.347022,-4.258348,-3.587544,...,4.367099,-0.078792,-1.947600,6.475665,-8.298667,-7.190138,2.670641,-3.040704,5.743192,2.828484
16,7.517741,-0.668608,-2.349386,0.576194,3.205817,3.418862,-5.238684,5.185432,-2.962818,-2.843899,...,6.128017,-0.476227,-2.057771,5.692659,-10.035450,-8.959109,0.893540,-3.915458,6.361079,2.199180


In [18]:
days_polars = df_polars.group_by_dynamic('timestamp', every='1d').agg(pl.exclude('timestamp').mean())

In [19]:
days_polars

timestamp,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,…,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-09-08 00:00:00,1.616301,1.300036,0.817093,-0.91363,0.535643,-0.429099,1.379245,1.250309,0.573959,1.237401,0.982781,1.159419,0.585748,2.077831,1.345749,1.448089,1.339835,1.6741,1.16905,1.125851,1.4459,1.007367,1.406774,1.651724,2.013898,0.903761,1.686244,2.16086,0.48558,0.60199,0.911072,1.105152,1.249857,2.314174,-0.020963,1.621261,…,1.973455,0.193607,-0.228364,2.240416,0.523938,-0.060897,2.703007,0.729973,1.013803,1.009531,0.740613,1.451533,2.310108,1.665348,0.983064,0.342018,0.904942,1.264219,1.118301,1.154367,0.917321,1.626718,0.423192,0.455271,0.949298,1.498886,0.752201,1.028077,0.495149,0.427012,1.460489,1.121102,0.547403,0.544154,0.869386,0.418865,0.095361
2024-09-09 00:00:00,1.786842,-0.056498,1.201573,-3.228591,0.872878,-1.87554,0.358851,1.010238,0.715874,0.827519,2.091645,1.052777,-0.118401,1.590183,2.055889,0.140329,2.170482,2.159051,1.638223,1.479489,2.707478,2.720873,2.553631,0.043101,2.904378,1.331285,2.855481,3.873791,1.276888,0.344223,3.335522,0.844653,1.134549,2.265679,0.05006,2.27837,…,0.460976,0.410341,0.108861,4.748316,1.627707,0.539094,3.540375,-0.552004,3.050409,0.69322,2.125061,0.401773,3.052611,1.326555,1.08147,-2.080009,-0.369088,0.746834,3.414918,0.333866,0.105144,-0.072555,-0.910073,-0.06145,0.458449,2.09503,1.380138,-0.706015,-1.444457,2.04932,3.110629,0.452043,0.017991,1.616743,2.704365,1.580477,0.372258
2024-09-10 00:00:00,1.166582,-1.765113,2.616493,-4.584708,0.898354,-1.781852,-1.061848,3.12399,-1.051368,0.092403,2.106916,-0.72748,0.498398,1.416709,0.363319,-0.391991,3.839071,-0.559622,1.432966,2.574934,3.257619,3.773844,1.94342,0.487923,4.609183,0.432212,2.489001,5.194257,1.394994,-1.117632,5.396862,1.058438,-0.544048,3.556528,-0.427315,2.231885,…,-2.881626,0.569615,-1.261293,5.996657,2.038671,1.031554,4.983702,1.927114,3.448345,0.929329,1.797745,1.851652,5.041373,2.039314,3.29611,-1.226162,0.158645,-1.090539,4.955187,-0.847299,2.466571,-0.566162,-0.115582,-1.271779,-0.900249,1.29768,-0.05552,-0.666805,-0.931251,5.258575,4.48965,0.336976,-2.257478,2.266737,3.005411,0.266224,0.728347
2024-09-11 00:00:00,0.684858,0.075567,3.814915,-5.174099,-1.534156,-0.894954,-2.950941,6.056719,-2.593477,0.549919,3.437001,-2.011032,1.015086,1.589247,-1.553823,-1.732432,5.557026,-0.534163,1.742049,2.292838,2.839152,2.494044,4.129099,3.513298,5.858024,0.533401,3.977967,3.367904,0.175142,-1.95852,5.924509,3.221549,-2.133666,2.881073,-0.571055,0.948886,…,-4.786809,1.142252,-3.188121,6.507497,-0.666797,0.535986,4.66567,3.929653,2.002436,2.061943,1.742559,3.647125,6.660319,0.926644,3.59704,-0.11981,0.077828,-3.857305,6.467044,-2.50863,2.708966,1.772397,0.719615,-1.489783,-2.405882,0.509248,-1.610418,-0.150487,-0.922968,4.638803,6.565021,-2.114639,-2.469373,2.944349,1.892101,0.34317,-0.24744
2024-09-12 00:00:00,1.235231,1.158072,3.888712,-3.935502,-1.39885,0.48998,-3.206239,6.448509,-3.039776,-2.741378,3.292258,-1.328935,1.091565,3.241458,-1.942603,-2.442468,4.576457,-0.607023,3.264525,2.481576,3.089234,1.256887,4.097257,4.340677,4.585012,0.637519,4.171094,0.218176,-3.239449,0.180732,4.846397,4.185399,-2.495979,3.723028,2.443621,-0.20868,…,-3.441645,1.108775,-1.878083,6.426544,-2.17253,2.38261,4.544575,5.203969,1.507189,3.275772,1.413667,3.411534,6.781933,0.198718,2.15762,1.793607,-1.232916,-4.271413,7.815731,-3.907045,3.654803,3.66498,1.935722,-0.288148,-3.070293,-0.746518,-1.875368,2.216628,-0.64744,3.961334,8.398948,-4.38

In [26]:
%timeit days_pandas = df_pandas.groupby(df_pandas.timestamp.dt.day).mean().drop('timestamp', axis=1)

612 ms ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit days_polars = df_polars.group_by_dynamic('timestamp', every='1d').agg(pl.exclude('timestamp').mean())

130 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit df_pandas['avg'] = df_pandas[[f'C{i}' for i in range(1, 101)]].sum(axis=1)

255 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit df_polars.select(pl.all(), pl.mean_horizontal(pl.exclude('timestamp')).alias('avg'))

54.5 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


On these operations, polars is faster as well.